## A* algorithm

In [ ]:
# here goes the code for A* algorithm
# add extra explanations in markdown

## Dijkstra

In [ ]:
# here goes the code for Dijkstra algorithm
# add extra explanations in markdown